In [1]:
import pandas as pd
import numpy as np

In [2]:
#Importar as dimensoes que vao entrar na tabela de factos 1

df1=pd.read_csv('DimData.csv')
df2=pd.read_csv('DimLocalizacao.csv')
df3=pd.read_csv('DimEurovisao.csv')
df4=pd.read_csv('DimJunk.csv')
df5=pd.read_excel('Vizinhos.xlsx', sheet_name='LB_Eurovision')
dfjunk=pd.read_csv('DimJunk.csv')
dfbase=pd.read_excel('Eurovisao.xlsx')
dffactos1=pd.DataFrame()

In [3]:
#Merge com a dimData
dfbase = pd.merge(dfbase, df1, how='inner', left_on = 'Year', right_on = 'Ano')

#Merge com a dimEurovisao utilizando o ano pois nao existia nenhum outra ligacao da eurovisao nesta tabela
dfbase = pd.merge(dfbase, df3, how='inner', left_on = 'Year', right_on = 'Ano')

#Apagar colunas que nao irao ser utilizadas
dfbase=dfbase.drop(['Host City', 'Ano_x', 'MetadeDaDecada','VerificacaoTelevoto','DescricaoPontos','Decada', 'Seculo', 
                    'TotalParticipantes','VerificacaoVotoJuri','TotalPontosPaisDa','MaxPontosPaisADaPaisB',
                    'TotalPontosDisponivelConcurso','NumeroEdicao', 'MaxPontuacaoPais', 'Ano_y','PaisAnfitriao',
                    'CidadeAnfitria','RegraLinguagem'],axis=1)

#Merge com a dimLocalizacao
dfbase = pd.merge(dfbase, df2, how='outer', left_on = 'From', right_on = 'Pais')

#Alteracao do nome da coluna para possibilitar a existencia de duas chaves estrangeiras 
dfbase.rename(columns = {'IDLocalizacao':'IDLocalizacaoDa'}, inplace = True)

#Apagar colunas que nao irao ser utilizadas
dfbase=dfbase.drop(['Pais','Regiao','Continente','NumeroDeVizinhos'],axis=1)

#Segundo merge com a dimLocalizacao e alteracao do nome da coluna
dfbase = pd.merge(dfbase, df2, how='outer', left_on = 'To', right_on = 'Pais')
dfbase.rename(columns = {'IDLocalizacao':'IDLocalizacaoRecebe'}, inplace = True)

#Apagar colunas que nao irao ser utilizadas
dfbase=dfbase.drop(['Pais','Regiao','Continente','NumeroDeVizinhos'],axis=1)

#Juncao com a tabela de vizinhos
dfbase = pd.merge(dfbase, df5, how='outer', left_on = 'From', right_on = 'Name')

#Procura de paises vizinhos
conditions2=[(dfbase['To']==dfbase['Neighbour1']),
             (dfbase['To']==dfbase['Neighbour2']),
             (dfbase['To']==dfbase['Neighbour3']),
             (dfbase['To']==dfbase['Neighbour4']),
             (dfbase['To']==dfbase['Neighbour5']),
             (dfbase['To']==dfbase['Neighbour6']),
             (dfbase['To']==dfbase['Neighbour7']),
             (dfbase['To']==dfbase['Neighbour8']),
             (dfbase['To']==dfbase['Neighbour9']),
             (dfbase['To']==dfbase['Neighbour10'])]

values2=['PaisesVizinhos','PaisesVizinhos','PaisesVizinhos','PaisesVizinhos','PaisesVizinhos',
         'PaisesVizinhos','PaisesVizinhos','PaisesVizinhos','PaisesVizinhos','PaisesVizinhos']
dfbase['VerifVizinho']=np.select(conditions2,values2)

#definir todos os vazios da coluna como paises nao vizinhos
vizinho_col = dfbase['VerifVizinho']
vizinho_col.replace(to_replace = '0', value = 'PaisesNaoVizinhos', inplace=True)

#Alteracao de valores na tabela Points type para fazer juncao com a Junk Dimension
ptype_col = dfbase['Points type']
ptype_col.replace(to_replace = 'Points given by televoters', value = 'Televoto', inplace=True)
ptype_col.replace(to_replace = 'Points given by the jury', value = 'VotoJuri', inplace=True)
ptype_col.replace(to_replace = 'Points given', value = 'VotoNaoDiscriminado', inplace=True)

#Mudanca do tipo de dados das colunas para permitir o merge dos dois dataframes
dfbase['Points type'] = dfbase['Points type'].astype(str)
dfbase['VerifVizinho'] = dfbase['VerifVizinho'].astype(str)
dfjunk['VerificacaoVizinho']=dfjunk['VerificacaoVizinho'].astype(str)
dfjunk['VerificacaoVoto']=dfjunk['VerificacaoVoto'].astype(str)

#Merge com a JunkDimension
dfbase = pd.merge(dfbase, dfjunk, how='left', left_on = ['Points type','VerifVizinho'], 
                  right_on = ['VerificacaoVoto','VerificacaoVizinho'])

dfbase=dfbase.drop(dfbase.index[13867:13873])
print(dfbase)

#Copiar as colunas relevantes para a nova tabela de factos
dffactos1['IDData']=dfbase['IDData'].astype(int)
dffactos1['IDEurovisao']=dfbase['IDEurovisao'].astype(int)
dffactos1['IDLocalizacaoDa']=dfbase['IDLocalizacaoDa'].astype(int)
dffactos1['IDLocalizacaoRecebe']=dfbase['IDLocalizacaoRecebe'].astype(int)
dffactos1['IDJunk']=dfbase['IDJunk'].astype(int)
dfbase['Points'] = dfbase['Points'].astype(int)
dffactos1['Pontos']=dfbase['Points']

#Exportar resultado final para csv
dffactos1.to_csv("FactTable1_final.csv", index=False)

          ID_x    Year          Points type                   From       To  \
0          0.0  2015.0  VotoNaoDiscriminado               Slovenia   Sweden   
1        432.0  2014.0  VotoNaoDiscriminado               Slovenia   Sweden   
2       1196.0  2011.0  VotoNaoDiscriminado               Slovenia   Sweden   
3       3074.0  2003.0  VotoNaoDiscriminado               Slovenia   Sweden   
4       3273.0  2002.0  VotoNaoDiscriminado               Slovenia   Sweden   
...        ...     ...                  ...                    ...      ...   
13862      NaN  2004.0  VotoNaoDiscriminado  Serbia and Montenegro  Albania   
13863  11179.0  2005.0  VotoNaoDiscriminado  Serbia and Montenegro   Turkey   
13864      NaN  2004.0  VotoNaoDiscriminado  Serbia and Montenegro  Estonia   
13865      NaN  2004.0  VotoNaoDiscriminado  Serbia and Montenegro   Israel   
13866  11178.0  2005.0  VotoNaoDiscriminado  Serbia and Montenegro  Romania   

       Points  Unnamed: 7  Unnamed: 8  Unnamed: 9  

In [4]:
dffactos1.dtypes

IDData                 int32
IDEurovisao            int32
IDLocalizacaoDa        int32
IDLocalizacaoRecebe    int32
IDJunk                 int32
Pontos                 int32
dtype: object